# [실습] LangChain Expression Language


LangChain Expression Language(LCEL)는 랭체인에서 체인을 간결하게 구성하는 문법입니다.   
먼저, LCEL에서 체인이 구성되는 기본적인 구조에 대해 알아봅시다.


In [1]:
!pip install openai langchain langchain_openai -q

In [1]:
import os
import json

with open("api_key.json", "r") as f:
    config = json.load(f)

api_key = config["OPENAI_API_KEY"]

#os.environ["OPENAI_API_KEY"] = "<OpenAI_API의 API 키>"
os.environ["OPENAI_API_KEY"] = api_key


기존에는 `LLM.invoke(prompt.format(___))` 형태로 실행을 했었는데요.    
LCEL의 가장 큰 특징은, Chain의 구성 요소를 **|**  (파이프)로 연결하여 한 번에 실행한다는 점입니다. 예시를 보겠습니다.

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


# topic에 대한 영어 농담을 하고, 이것이 왜 농담인지 한국어로 설명하세요.
fun_chat_template = ChatPromptTemplate.from_messages([
    ('user',"""tell me an English joke about {topic},
also, explain in Korean why it is fun for english-speakers.
Include translation of the joke.""")
])


llm=ChatOpenAI(temperature=0.5, model='gpt-4o-mini',max_tokens=1000)
# LLM 유머는 지능에 비례



In [3]:
response = llm.invoke(fun_chat_template.format_messages(topic='AI'))
response

AIMessage(content='**Joke:**\nWhy did the AI go broke?  \nBecause it lost its cache!\n\n**Translation:**\n왜 AI가 파산했을까요?  \n왜냐하면 캐시를 잃어버렸기 때문이에요!\n\n**Explanation in Korean:**\n이 농담은 영어 사용자들에게 재미있습니다. "cache"라는 단어는 두 가지 의미를 가지고 있습니다. 첫 번째는 컴퓨터에서 데이터를 빠르게 접근하기 위해 저장하는 메모리의 일종입니다. 두 번째는 "cash"와 발음이 비슷하여 돈을 의미합니다. 그래서 AI가 "cache"를 잃어버렸다는 것은 데이터 저장소를 잃어버린 것과 동시에 돈을 잃었다는 이중적인 의미를 가지고 있어 웃음을 줍니다. 영어 발음의 유사성과 기술적 개념이 결합되어 있어 더욱 재미있습니다.', response_metadata={'token_usage': {'completion_tokens': 187, 'prompt_tokens': 35, 'total_tokens': 222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-113620fd-b450-4362-8938-d4cba6a203ff-0')

In [4]:
print(response.content)

**Joke:**
Why did the AI go broke?  
Because it lost its cache!

**Translation:**
왜 AI가 파산했을까요?  
왜냐하면 캐시를 잃어버렸기 때문이에요!

**Explanation in Korean:**
이 농담은 영어 사용자들에게 재미있습니다. "cache"라는 단어는 두 가지 의미를 가지고 있습니다. 첫 번째는 컴퓨터에서 데이터를 빠르게 접근하기 위해 저장하는 메모리의 일종입니다. 두 번째는 "cash"와 발음이 비슷하여 돈을 의미합니다. 그래서 AI가 "cache"를 잃어버렸다는 것은 데이터 저장소를 잃어버린 것과 동시에 돈을 잃었다는 이중적인 의미를 가지고 있어 웃음을 줍니다. 영어 발음의 유사성과 기술적 개념이 결합되어 있어 더욱 재미있습니다.


-----------
LCEL의 인터페이스는 조금 다릅니다.

템플릿과 llm 모델을 설정하고, 이를 하나로 묶어 체인을 생성합니다.

In [5]:
joke = fun_chat_template | llm
# shift + \ = |||||
joke

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me an English joke about {topic},\nalso, explain in Korean why it is fun for english-speakers.\nInclude translation of the joke.'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f1c02bdf6d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f1c05c3d690>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1000)

이후, 체인의 invoke를 실행하며 입력 포맷을 전달하면, 순서대로 체인이 실행되며 최종 결과로 연결됩니다.    
입력 포맷은 Dict 형식으로 전달합니다.

In [6]:
response = joke.invoke({'topic':'apple'})
# invoke에 대한 입력은 딕셔너리로 처리 ':'
# 단, 매개 변수가 1개일 때는
# joke.invoke('apple')과 같이 문자열로 입력할 수도 있음
response

AIMessage(content="**Joke:**\nWhy did the apple stop in the middle of the road?  \nBecause it ran out of juice!\n\n**Translation:**\n사과가 왜 도로 한가운데서 멈췄나요?  \n즉석에서 주스가 떨어졌기 때문이에요!\n\n**Explanation in Korean:**\n이 농담은 '주스(juice)'라는 단어의 이중적인 의미 때문에 재미있습니다. 영어에서 '주스'는 과일 주스를 의미할 뿐만 아니라, '힘'이나 '에너지'를 의미하는 속어로도 사용됩니다. 사과가 도로에서 멈춘 이유가 주스가 떨어졌기 때문에라는 설정은 사과가 실제로는 물리적인 힘이 없어서 멈춘 것처럼 들리게 하여 웃음을 유발합니다. 이런 언어 유희는 영어 사용자들에게 특히 재미있게 느껴집니다.", response_metadata={'token_usage': {'completion_tokens': 194, 'prompt_tokens': 35, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-49d1dc82-ea8a-4f96-bd3a-7fee7505f6b0-0')

In [7]:
print(response.content)

**Joke:**
Why did the apple stop in the middle of the road?  
Because it ran out of juice!

**Translation:**
사과가 왜 도로 한가운데서 멈췄나요?  
즉석에서 주스가 떨어졌기 때문이에요!

**Explanation in Korean:**
이 농담은 '주스(juice)'라는 단어의 이중적인 의미 때문에 재미있습니다. 영어에서 '주스'는 과일 주스를 의미할 뿐만 아니라, '힘'이나 '에너지'를 의미하는 속어로도 사용됩니다. 사과가 도로에서 멈춘 이유가 주스가 떨어졌기 때문에라는 설정은 사과가 실제로는 물리적인 힘이 없어서 멈춘 것처럼 들리게 하여 웃음을 유발합니다. 이런 언어 유희는 영어 사용자들에게 특히 재미있게 느껴집니다.


## 실습) 매개변수가 2개인 Prompt-LLM Chain 생성하기   
임의의 ChatPromptTemplate를 만들고, 2개의 매개변수를 받도록 구성하여 체인을 만들고 실행하세요.

In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


# topic에 대한 영어 농담을 하고, 이것이 왜 농담인지 한국어로 설명하세요.
fun_chat_template1 = ChatPromptTemplate.from_messages([
    ('user',"""tell me an English joke about {topic},
also, explain in Korean why it is fun for english-speakers.
Include translation of the joke.""")
])

fun_chat_template2 = ChatPromptTemplate.from_messages([
    ('user',"""tell me an Korean joke about {topic},
also, explain in Korean why it is fun for korean-speakers.
Include translation of the joke.""")
])


llm=ChatOpenAI(temperature=0.5, model='gpt-4o-mini',max_tokens=1000)
# LLM 유머는 지능에 비례

In [9]:
response1 = llm.invoke(fun_chat_template1.format_messages(topic='AI'))
response2 = llm.invoke(fun_chat_template2.format_messages(topic='AI'))

print(response1.content)
print(response2.content)

**Joke:**
Why did the AI go broke?  
Because it couldn't find a good "cache" flow!

**Translation:**
AI가 왜 파산했나요?  
왜냐하면 좋은 "캐시" 흐름을 찾을 수 없었기 때문이에요!

**Explanation in Korean:**
이 농담은 영어에서 "cache"라는 단어의 이중 의미를 이용하고 있습니다. "Cache"는 컴퓨터에서 데이터를 빠르게 저장하고 접근하기 위한 메모리의 일종을 의미하는데, 발음이 "cash"와 비슷합니다. "Cash flow"는 사업의 현금 흐름을 의미하죠. 그러므로 AI가 "cache" 흐름을 찾지 못했다는 것은 실제로는 돈이 부족하다는 의미로 해석될 수 있어, 언어 유희가 재미를 더합니다. 영어 사용자들은 이러한 언어적 장치를 통해 웃음을 느낄 수 있습니다.
**Korean Joke:**

왜 인공지능은 항상 웃고 있을까요?  
왜냐하면 "데이터"를 많이 가지고 있어서 "유머"가 넘치거든요!

**Translation:**

Why is AI always smiling?  
Because it has a lot of "data," so it overflows with "humor"!

**Explanation in Korean:**

이 농담은 한국어 사용자에게 재미있는 이유는 두 가지입니다. 첫째, "데이터"와 "유머"라는 단어의 발음이 비슷해서 언어 유희가 생깁니다. 둘째, 인공지능(AI)에 대한 일반적인 인식과 기대를 반영하여, AI가 많은 정보를 가지고 있기 때문에 유머 감각이 뛰어나다는 아이러니한 상황을 제시합니다. 한국어 사용자들은 이러한 언어적 유희와 아이러니를 통해 웃음을 느낄 수 있습니다.


In [10]:
joke = fun_chat_template1 | fun_chat_template2 | llm
# shift + \ = |||||
print(joke)


first=ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me an English joke about {topic},\nalso, explain in Korean why it is fun for english-speakers.\nInclude translation of the joke.'))]) middle=[ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me an Korean joke about {topic},\nalso, explain in Korean why it is fun for korean-speakers.\nInclude translation of the joke.'))])] last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f1bfa848be0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f1bfa84afe0>, model_name='gpt-4o-mini', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1000)


LCEL의 체인에는 **파서(Parser)** 를 추가할 수 있습니다.    
파서는 출력 형식을 변환합니다.

StrOutputParser : 출력 결과를 String 형식으로 변환합니다.

In [4]:
from langchain_core.output_parsers import StrOutputParser

recipe_template=ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 조리법을 아는 쉐프입니다.'),
    ('user','저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!')
])

In [12]:
# StrOutputParser를 사용하여 출력 형식을 str로 고정, 원래는 딕셔너리로 나와서 .content를 해줘야 했음.
recipe_chain = recipe_template | llm | StrOutputParser()
response = recipe_chain.invoke({'ingredient':'콜라'})
print(response)

콜라를 이용한 환상적인 외국 음식으로는 "콜라 바비큐 소스"를 활용한 바비큐 요리를 추천합니다. 이 요리는 미국식 바비큐에서 많이 사용되며, 콜라의 단맛과 탄산이 고기의 풍미를 더욱 살려줍니다.

### 콜라 바비큐 소스 레시피

#### 재료
- 콜라 1컵
- 케첩 1컵
- 간장 1/4컵
- 사과 식초 1/4컵
- 갈색 설탕 1/2컵
- 마늘 가루 1작은술
- 양파 가루 1작은술
- 후추 1/2작은술
- 훈제 파프리카 1작은술 (선택 사항)

#### 만드는 법
1. **소스 만들기**: 중간 크기의 냄비에 콜라, 케첩, 간장, 사과 식초, 갈색 설탕, 마늘 가루, 양파 가루, 후추, 훈제 파프리카를 넣고 잘 섞어주세요.
2. **조리**: 냄비를 중불에 올리고 끓기 시작하면 불을 줄여서 약 20-30분간 끓입니다. 소스가 걸쭉해질 때까지 끓여주세요.
3. **고기 준비**: 돼지고기, 닭고기, 또는 소고기 등 원하는 고기를 준비합니다. 고기에 소금을 뿌리고, 원하는 경우 후추나 다른 향신료로 간을 해주세요.
4. **구이**: 그릴이나 오븐에서 고기를 구우면서 중간중간 콜라 바비큐 소스를 발라줍니다. 고기가 잘 익고 바삭해질 때까지 구워주세요.
5. **서빙**: 구운 고기를 접시에 담고 남은 소스를 곁들여 제공하면 완성입니다.

이 요리는 파티나 바비큐 모임에 매우 인기가 있으며, 콜라의 달콤한 맛이 고기의 풍미와 잘 어우러집니다. 즐겁게 요리해보세요!


파서는 스트링이 아닌 json 형식으로도 만들 수 있습니다.   
프롬프트에서 형식을 요청하고, 이를 파서와 결합하여 변환하는 방식입니다.

In [5]:
from langchain_core.output_parsers import JsonOutputParser

jsonparser = JsonOutputParser()

In [14]:
jsonparser

JsonOutputParser()

In [15]:
jsonparser.get_format_instructions()

'Return a JSON object.'

In [16]:
recipe_template=ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 환상적인 파인 다이닝을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요.''')
])

recipe_chain = recipe_template | llm | jsonparser


In [17]:
response = recipe_chain.invoke({'ingredient':'콜라'})
response

{'recipe_name': '콜라 글레이즈드 스테이크',
 'servings': 4,
 'ingredients': {'steak': {'type': 'ribeye',
   'amount': '4 pieces',
   'weight': '200g each'},
  'cola': {'amount': '1 cup'},
  'soy_sauce': {'amount': '1/4 cup'},
  'brown_sugar': {'amount': '2 tablespoons'},
  'garlic': {'amount': '3 cloves', 'preparation': 'minced'},
  'ginger': {'amount': '1 tablespoon', 'preparation': 'grated'},
  'olive_oil': {'amount': '2 tablespoons'},
  'salt': {'amount': 'to taste'},
  'black_pepper': {'amount': 'to taste'},
  'vegetable_side': {'type': 'asparagus', 'amount': '200g'}},
 'instructions': ['1. 콜라, 간장, 갈색 설탕, 다진 마늘, 간 생강을 작은 냄비에 넣고 중불에서 끓입니다.',
  '2. 소스가 반으로 줄어들고 걸쭉해지면 불에서 내리고 식힙니다.',
  '3. 스테이크에 소금과 후추로 간을 합니다.',
  '4. 팬에 올리브 오일을 두르고 중불에서 스테이크를 양면이 갈색이 될 때까지 굽습니다.',
  '5. 스테이크가 거의 다 익었을 때, 준비한 콜라 소스를 스테이크 위에 발라줍니다.',
  '6. 스테이크를 완전히 익힐 때까지 조리합니다.',
  '7. 아스파라거스를 소금물에 살짝 데쳐서 곁들입니다.',
  '8. 스테이크를 접시에 담고 아스파라거스를 옆에 배치한 후 남은 콜라 소스를 뿌려서 서빙합니다.'],
 'cooking_time': {'preparation': '10 minutes',
  'cook

In [19]:
response['recipe_name']

'콜라 글레이즈드 스테이크'

Json으로 파싱하는 방법은 활용도가 높지만, 실행할 때마다 결과뿐만 아니라 형식도 달라진다는 문제가 있습니다.

In [20]:
response = recipe_chain.invoke({'ingredient':'콜라'})
response

{'recipe': {'name': '콜라 글레이즈드 스테이크',
  'cuisine': '퓨전',
  'servings': 2,
  'ingredients': [{'item': '소고기 스테이크', 'quantity': '2', 'unit': '조각'},
   {'item': '콜라', 'quantity': '1', 'unit': '컵'},
   {'item': '간장', 'quantity': '2', 'unit': '큰술'},
   {'item': '올리브 오일', 'quantity': '1', 'unit': '큰술'},
   {'item': '다진 마늘', 'quantity': '2', 'unit': '쪽'},
   {'item': '신선한 생강', 'quantity': '1', 'unit': '작은 조각'},
   {'item': '후추', 'quantity': '적당량', 'unit': ''},
   {'item': '소금', 'quantity': '적당량', 'unit': ''},
   {'item': '신선한 허브 (타임 또는 로즈마리)', 'quantity': '약간', 'unit': ''}],
  'instructions': ['1. 스테이크에 소금과 후추로 간을 하고, 실온에서 30분 정도 재워둡니다.',
   '2. 팬에 올리브 오일을 두르고 중불로 가열합니다.',
   '3. 스테이크를 팬에 넣고 양면을 각각 3-4분간 구워서 원하는 익힘 정도로 조리합니다.',
   '4. 스테이크를 팬에서 꺼내고 따로 두어 휴식시킵니다.',
   '5. 같은 팬에 다진 마늘과 생강을 넣고 볶아 향을 내고, 그 후 콜라와 간장을 추가합니다.',
   '6. 소스를 중불에서 끓여 농도가 걸쭉해질 때까지 약 10분간 조리합니다.',
   '7. 스테이크를 다시 팬에 넣고 소스를 발라가며 1-2분간 더 익힙니다.',
   '8. 접시에 스테이크를 담고, 위에 소스를 뿌린 후 신선한 허브로 장식합니다.',
   '9. 원하는 사이드와 함께 서빙합니다.'],
  

## Pydantic을 이용해 확실한 형식 지정하기

pydantic은 데이터 형식에 제약조건을 두고 이를 준수하는지 검증하는 라이브러리입니다.


In [6]:
from pydantic import BaseModel, Field
# pydantic 연동

class Recipe(BaseModel):
    name: str = Field(description="음식 이름")
    # name: 문자열, 설명은 "음식 이름"
    difficulty: str = Field(description="만들기의 난이도")
    kick: str = Field(description='맛의 포인트')

    origin: str = Field(description="원산지")
    ingredients: list[str] = Field(description="재료")
    # ingredients: 문자열 리스트, 설명은 "재료"

    instructions: list[str] = Field(description="조리법")
    tip: str = Field(description='조리 과정 팁')


In [22]:
parser = JsonOutputParser(pydantic_object=Recipe)

In [23]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "\uc74c\uc2dd \uc774\ub984", "type": "string"}, "difficulty": {"title": "Difficulty", "description": "\ub9cc\ub4e4\uae30\uc758 \ub09c\uc774\ub3c4", "type": "string"}, "kick": {"title": "Kick", "description": "\ub9db\uc758 \ud3ec\uc778\ud2b8", "type": "string"}, "origin": {"title": "Origin", "description": "\uc6d0\uc0b0\uc9c0", "type": "string"}, "ingredients": {"title": "Ingredients", "description": "\uc7ac\ub8cc", "type": "array", "items": {"type": "string"}}, "instructions": {"title": 

In [24]:
# depreciated
# refined_json_instructions = parser.get_format_instructions().encode().decode('unicode_escape')
# print(refined_json_instructions)

In [25]:
recipe_template2 =ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.
     구조는 다음과 같이 해주세요.
     구조: {instruction}''')
])

recipe_chain2 = recipe_template2 | llm | parser


In [26]:
recipe_template2

ChatPromptTemplate(input_variables=['ingredient', 'instruction'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ingredient', 'instruction'], template='저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!\n    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.\n     구조는 다음과 같이 해주세요.\n     구조: {instruction}'))])

In [27]:
recipe_chain2.invoke({'ingredient':'생강', 'instruction':parser.get_format_instructions()}) #refined_json_instructions})

{'name': '생강 치킨 카레',
 'difficulty': '중간',
 'kick': '매콤하고 향긋한 맛',
 'origin': '인도',
 'ingredients': ['닭고기 500g',
  '양파 1개',
  '마늘 3쪽',
  '생강 2큰술',
  '토마토 1개',
  '카레 가루 2큰술',
  '코코넛 밀크 200ml',
  '식용유 2큰술',
  '소금 적당량',
  '후추 적당량',
  '고수 잎 (선택 사항)'],
 'instructions': ['닭고기를 한입 크기로 썰고, 소금과 후추로 밑간을 해둡니다.',
  '양파, 마늘, 생강을 잘게 다집니다.',
  '팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 익힙니다.',
  '마늘과 생강을 추가하고 향이 올라올 때까지 볶습니다.',
  '닭고기를 팬에 넣고 겉면이 노릇해질 때까지 볶습니다.',
  '다진 토마토와 카레 가루를 넣고 잘 섞어줍니다.',
  '코코넛 밀크를 추가하고 끓인 후 중불에서 20분 정도 익힙니다.',
  '고수 잎으로 장식하여 서빙합니다.'],
 'tip': '생강을 더 많이 넣으면 더 향긋한 맛을 즐길 수 있습니다.'}

# Structured Output
LangChain의 Structured_Output 기능을 사용할 수도 있습니다.

In [ ]:
!pip install pydantic=1.10.8

In [29]:
llm = ChatOpenAI(temperature=0.5, model='gpt-4o-mini',max_tokens=1000)

structured_llm = llm.with_structured_output(Recipe)
structured_llm.invoke("생강으로 만들 수 있는 요리 레시피 알려주세요.")

Recipe(name='생강차', difficulty='쉬움', kick='상큼하고 따뜻한 맛', origin='한국', ingredients=['생강', '꿀', '물', '레몬'], instructions=['생강을 얇게 썬다.', '냄비에 물을 끓인다.', '끓는 물에 생강을 넣고 10분 정도 끓인다.', '불을 끄고 꿀과 레몬을 넣어 잘 섞는다.', '따뜻하게 즐긴다.'], tip='생강은 신선한 것을 사용하면 더 향긋하다.')

In [30]:
structured_llm = llm.with_structured_output(Recipe, method='json_mode')
structured_llm.invoke("생강으로 만들 수 있는 요리 레시피 알려주세요."+parser.get_format_instructions())

Recipe(name='생강차', difficulty='쉬움', kick='상큼하고 따뜻한 맛', origin='한국', ingredients=['생강 100g', '물 1리터', '꿀 또는 설탕 (취향에 따라)', '레몬 (선택 사항)'], instructions=['생강을 깨끗이 씻고 껍질을 벗긴 후 얇게 썬다.', '냄비에 물을 붓고 썬 생강을 넣는다.', '중불에서 20-30분 정도 끓인다.', '불을 끄고, 원하는 만큼 꿀 또는 설탕을 넣어 잘 저어준다.', '차가운 물로 희석하거나 뜨겁게 마신다. 원한다면 레몬을 추가한다.'], tip='생강은 면역력 강화에 도움을 주며, 감기 예방에 효과적이다.')

이제, 댓글이 주어지면 댓글의 답변을 생성해 보겠습니다.

In [7]:
import pandas as pd
reviews = pd.read_csv('./data/1_1.data_reviews.csv')
print(reviews.shape)
reviews.head()

(50, 3)


,Num,Review,Label
0,1,그닥 맛있고 좋은 고기인지는 모르겠내요 테이블 나누어서 두팀 받는데 옆 테이블 시끄...,-1
1,2,적당히 좋은 소고기를 싸지 않은 가격에 맛볼 수 있다. 하지만 인기나 리뷰에 비해선...,-1
2,3,기름진맛 역시맛있네요 많이먹긴 힘들지만 맛있는 소고기집인건 틀림없어요,1
3,4,"지인이 추천하고 짬뽕맛집이래서 찾아갔는데, 주차도 골목길에 해야되고 맛도 별로네요",-1
4,5,매운단계별짬뽕과 불향가득짜장 직접만든소스가별미인탕수육 제입맛에는딱맞아서자주찾게되요^^*,1


In [8]:
reply_template = ChatPromptTemplate.from_messages([
    ('system','''당신은 레스토랑의 주인입니다.
고객이 다음과 같은 리뷰를 남겼을 때, 답변을 작성해 주세요.
첫 문장은 가상의 레스토랑 이름과 함께 인사하는 내용을 포함하세요.
고객의 의견에 매우 공감하여 답변하고, 부정적인 피드백은 사과하세요.
새로운 메뉴나 프로모션을 홍보호가, 재방문을 기원하세요.
밝고 유쾌한 톤으로 작성하고, 이모지를 매우 많이 추가하세요.

'''),
    ('user','''{review}''')

])

In [9]:
reply_chain = reply_template | llm | StrOutputParser()
reply_chain.invoke({'review':reviews['Review'][0]})

"안녕하세요! 🎉 저희 레스토랑 '맛있는 고기집'에 방문해 주셔서 정말 감사합니다! 😊\n\n고객님께서 말씀해 주신 부분에 대해 매우 공감합니다. 고기는 정말 중요한 요소인데, 기대에 미치지 못해 아쉬움을 드린 점 진심으로 사과드립니다. 😔 또한, 옆 테이블의 소음으로 인해 불편하셨다니 정말 안타깝습니다. 저희는 고객님께서 편안하게 식사하실 수 있도록 최선을 다하겠습니다! 💪\n\n다음 번에는 저희의 새로운 메뉴인 '특선 소고기 스테이크'를 꼭 한 번 시도해 보세요! 🍖✨ 그리고 매주 금요일에는 '고기 무한리필' 프로모션도 진행 중이니 많은 관심 부탁드립니다! 🎊\n\n고객님의 재방문을 진심으로 기다리고 있겠습니다! 항상 맛있고 즐거운 경험을 제공할 수 있도록 노력하겠습니다. 감사합니다! 🙏💖"

입력이 여러 개인 경우, 이를 리스트로 만들고 batch()를 이용해 한꺼번에 입력할 수 있습니다.

In [34]:
reviews.loc[:5,'Review'].to_list()

['그닥 맛있고 좋은 고기인지는 모르겠내요 테이블 나누어서 두팀 받는데 옆 테이블 시끄러워서 짜증이 많이 나내요',
 '적당히 좋은 소고기를 싸지 않은 가격에 맛볼 수 있다. 하지만 인기나 리뷰에 비해선 평범한 수준. 이런 맛과 가성비의 소고기집은 동네마다 아주아주 많다.',
 '기름진맛 역시맛있네요 많이먹긴 힘들지만 맛있는 소고기집인건 틀림없어요',
 '지인이 추천하고 짬뽕맛집이래서 찾아갔는데, 주차도 골목길에 해야되고 맛도 별로네요',
 '매운단계별짬뽕과 불향가득짜장 직접만든소스가별미인탕수육 제입맛에는딱맞아서자주찾게되요^^*',
 '신맛나지 않는 달콤한 탕수육도 맛있다 친절한 사장님도 추가 당분간 맛없는 배달짬뽕 먹을 생각하니 슬프다 사장님 쾌차하세요']

In [35]:
reply_chain.batch(reviews.loc[:5,'Review'].to_list())

["안녕하세요! 🌟 저희 레스토랑 '맛있는 고기집'에 방문해 주셔서 정말 감사드립니다! 😊\n\n고객님께서 말씀해 주신 부분에 대해 매우 공감합니다. 😔 고기 맛이 기대에 미치지 못했다니 정말 죄송합니다. 또한, 옆 테이블의 소음으로 불편함을 드린 점도 사과드립니다. 🙇\u200d♂️ 저희는 고객님들께서 편안하게 식사하실 수 있도록 최선을 다하고 있습니다!\n\n새로운 메뉴가 곧 출시될 예정이니 기대해 주세요! 🍽️✨ 그리고 특별 프로모션도 준비 중이니, 다음 방문 시 더 좋은 경험을 하실 수 있도록 노력하겠습니다! 🎉\n\n다시 한 번 소중한 피드백에 감사드리며, 다음에 꼭 다시 뵙기를 바랍니다! 💖 좋은 하루 되세요! 🌈",
 "안녕하세요! 🍽️ 저희 레스토랑 '소고기 천국'에 방문해 주셔서 감사합니다! 😊 고객님께서 말씀해 주신 내용에 깊이 공감합니다. 저희가 기대에 미치지 못한 점에 대해 진심으로 사과드립니다. 😔\n\n저희는 항상 최고의 소고기를 제공하기 위해 노력하고 있으며, 고객님의 소중한 피드백을 바탕으로 더욱 발전할 수 있도록 하겠습니다! 💪✨\n\n그리고 좋은 소식이 있어요! 🎉 저희는 곧 새로운 메뉴를 출시할 예정이며, 특별한 프로모션도 준비 중입니다! 🍖💖 다음 번 방문 시 기대하셔도 좋습니다! 고객님께서 다시 찾아주신다면 더욱 맛있고 특별한 경험을 드리도록 하겠습니다! 🌟\n\n다시 한 번 감사드리며, 곧 다시 뵙기를 기원합니다! 😊🍀",
 '안녕하세요! 🍽️ "맛있는 소고기집"에 오신 것을 환영합니다! 🎉 고객님께서 기름진 맛을 즐기셨다니 정말 기쁩니다! 😍 하지만 많이 드시기 힘드셨다니 아쉽네요. 😔 저희는 항상 고객님의 건강과 만족을 최우선으로 생각하고 있습니다! \n\n최근에 저희는 새로운 메뉴를 출시했답니다! 🍖✨ 다양한 소고기 요리와 함께 특별한 사이드 메뉴도 준비했으니, 다음 방문 때 꼭 한번 드셔보세요! 🎈 또한, 이번 주말에는 특별 프로모션이 있으니 기대하셔도 좋습니다! 🎊\n\n다시 한번 방문해 주시면 더욱 맛있는 경험

프롬프트를 잘 구성하거나, Schema를 사용한다면 답글의 형식을 통일할 수 있습니다.

<br><br>
## Runnables

Runnables는 LCEL의 기본 단위로, 입력을 받아 출력을 생성하는 기본 단위입니다.    
llm, prompt, chain 등이 모두 Runnable 구조에 해당합니다.

이번에는, 데이터 흐름을 제어하는 특별한 Runnable인   
RunnablePassthrough와 RunnableParallel을 이용해 체인을 구성해 보겠습니다.


<br><br>
### RunnablePassthrough
RunnablePassthrough는 체인의 직전 출력을 그대로 가져옵니다.

In [10]:
from langchain.schema.runnable import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "{director}의 대표 작품은 무엇입니까?")
chain1 = (
    prompt1
    | llm
    | StrOutputParser()
    | {'answer': RunnablePassthrough()})

response = chain1.invoke("스티븐 스필버그")
print(response)

{'answer': "스티븐 스필버그는 영화 역사에서 가장 영향력 있는 감독 중 한 명으로, 그의 대표작은 여러 편이 있습니다. 주요 작품으로는 다음과 같은 영화들이 있습니다:\n\n1. **죠스 (Jaws, 1975)** - 스릴러와 호러의 요소가 결합된 이 영화는 여름 블록버스터의 시작을 알린 작품입니다.\n2. **이티 (E.T. the Extra-Terrestrial, 1982)** - 외계 생명체와 어린 소년의 우정을 그린 감동적인 이야기로, 많은 사랑을 받았습니다.\n3. **인디아나 존스 시리즈 (Indiana Jones series)** - 모험과 액션이 가득한 이 시리즈는 고고학자 인디아나 존스의 이야기를 다룹니다.\n4. **쉰들러 리스트 (Schindler's List, 1993)** - 홀로코스트를 배경으로 한 이 영화는 강력한 메시지를 전달하며, 많은 찬사를 받았습니다.\n5. **라이언 일병 구하기 (Saving Private Ryan, 1998)** - 제2차 세계대전을 배경으로 한 전쟁 영화로, 사실적인 전투 장면이 인상적입니다.\n6. **쿵푸 팬더 (Minority Report, 2002)** - 미래를 예측하는 경찰의 이야기를 다룬 사이언스 픽션 영화입니다.\n7. **링킹 파크 (Lincoln, 2012)** - 아브라함 링컨 대통령의 생애를 다룬 작품으로, 다니엘 데이 루이스의 연기가 돋보입니다.\n\n이 외에도 스필버그는 많은 다양한 장르의 영화를 제작해왔으며, 그의 작품은 전 세계적으로 큰 영향을 미쳤습니다."}


<br><br>
### RunnableParallel

RunnableParallel은 서로 다른 체인을 병렬적으로 실행합니다.

In [11]:
from langchain_core.runnables import RunnableParallel

prompt1 = ChatPromptTemplate.from_template( # 괄호 뒤에서 줄바꿈하면 다음 줄도 같은 줄로 인식
    "색깔을 하나 알려주세요, 색깔만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "음식을 하나 알려주세요, 음식만 출력하세요.")

chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()

chain3 = RunnableParallel(color = chain1, food = chain2)

chain3.invoke({}) # chain3.invoke()는 에러


{'color': '파란색', 'food': '김치찌개'}

<br><br><br>이번에는 LLM의 결과를 다음 LLM으로 연결해 보겠습니다.

In [12]:
prompt1 = ChatPromptTemplate.from_template("잭슨빌은 어느 나라의 도시입니까? 나라이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{country}의 대표적인 인물 3명을 나열하세요. 인물의 이름만 출력하세요."
)

chain1 = prompt1 | llm | StrOutputParser()
chain2 =(
    {"country": chain1} | prompt2 | llm | StrOutputParser()
)
chain2.invoke({})

'조지 워싱턴, 에이브러햄 링컨, 마틴 루터 킹 주니어'

RunnableParallel을 사용하면 chain1의 결과와 chain2의 결과를 함께 얻을 수 있습니다.   
이 때, chain1의 실행 결과를 chain2에 전달하는 방식으로 실행됩니다.

In [13]:
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()  # chain2 형식이 바뀜

chain3 = RunnableParallel(country = chain1).assign(people = chain2)

chain3.invoke({})

{'country': '미국', 'people': '조지 워싱턴, 에이브러햄 링컨, 마틴 루서 킹 주니어.'}

## 실습) 체인 연결하기

위 코드들을 참고하여, LLM을 3개 연결해서 3개 결과를 출력해 보세요.   
체인 3개로 만들 수도 있고, RunnableParallel을 사용할 수도 있습니다.   

예시)    
1. 영화->감독
2. 감독->가장 유명한 배우
3. 배우->상을 받은 영화

In [14]:
# 예시 프롬프트
prompt1 = ChatPromptTemplate.from_template("{movie}의 감독은 누구입니까? 감독 이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template("{director}와 작업한 가장 유명한 배우는 누구인가요? 배우 이름만 출력하세요.")
prompt3 = ChatPromptTemplate.from_template("{actor}는 무슨 영화로 상을 받았나요?")



In [15]:
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()
chain3 = prompt3 | llm | StrOutputParser()

chain4 = RunnableParallel(director = chain1).assign(actor = chain2).assign(award = chain3)

chain4.invoke('죠스')


{'director': '스티븐 스필버그',
 'actor': '리하네 모레노',
 'award': '리하네 모레노는 영화 "웨스트 사이드 스토리" (West Side Story, 2021)에서의 역할로 주목받았습니다. 이 영화는 스티븐 스필버그 감독의 작품으로, 그녀는 아나 역할을 맡아 뛰어난 연기력을 보여주었습니다. 이로 인해 여러 영화제에서 상을 수상하거나 후보에 오르는 성과를 거두었습니다. 특히, 그녀는 2022년 아카데미 시상식에서 여우조연상 후보에 올랐습니다.'}

<br><br>
chain2에서 새로운 매개변수가 추가되는 경우는 어떻게 해야 할까요?

Lambda 함수를 통해, 입력 dict로부터 값을 선택합니다.

In [19]:
prompt1 = ChatPromptTemplate.from_template("{city}는 어느 나라의 도시인가요? 나라 이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{country}의 유명한 인물은 누가 있나요? {num} 명의 이름을 나열하세요. 사람 이름만 ,로 구분하여 나열하세요."
)

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    RunnableParallel(country = chain1, num = lambda x:x['num'])
    # lambda x:f(x) --> x를 입력 받으면 f(x)를 return, x는 직전에 입력된 값
    # invoke에서 주어지는 dict를 전처리하기
    | prompt2
    | llm
    | StrOutputParser()
)

print(chain2.invoke({"city": "잭슨빌", "num": "3"}))

조지 워싱턴, 마틴 루터 킹 주니어, 앤젤리나 졸리


<br><br>
체인을 분리하고 RunnableParallel을 이용하면 중간 과정을 모두 출력할 수 있습니다.

In [17]:
chain4 = (prompt2
    | llm
    | StrOutputParser())

chain3 = RunnableParallel(country = chain1, num = lambda x:x['num']).assign(res = chain4)

chain3.invoke({"city": "코펜하겐", "num": "3"})

{'country': '덴마크', 'num': '3', 'res': '한스 크리스티안 안데르센, 소렌 키르케고르, 니콜라이 부르크그레비.'}

<br><br><br>JsonOutputParser를 쓴다면 아래와 같이 만들 수도 있습니다.

In [20]:
from langchain_core.output_parsers import JsonOutputParser

prompt1 = ChatPromptTemplate.from_template("미국 영화 배우와 대표작을 하나 나열하세요. json 형식으로 출력하고, 각 항목은 actor, movie로 표시하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{actor}는 {movie}에서 어떤 역할을 했습니까?"
)

chain1 = prompt1 | llm | JsonOutputParser()
chain2 =(
     chain1 | prompt2 | llm | StrOutputParser()
)
chain2.invoke({})

'Leonardo DiCaprio는 영화 "Inception"에서 주인공인 도미닉 "돔" 코브(Dominic "Dom" Cobb) 역할을 맡았습니다. 그는 꿈을 통해 사람의 무의식에 침투하여 정보를 훔치거나 아이디어를 심는 "추출" 전문가입니다. 영화는 그가 마지막 임무를 수행하기 위해 팀을 구성하고, 꿈의 세계에서 복잡한 상황에 맞서 싸우는 과정을 그립니다. DiCaprio의 연기는 이 영화에서 중요한 감정적 중심을 제공하며, 그의 캐릭터는 잃어버린 아내와의 관계와 그로 인한 내적 갈등을 다루고 있습니다.'